In [22]:
import nipype
import os

from nipype.interfaces.fsl import Info, MCFLIRT, FSLCommand, MeanImage
from nipype.interfaces.freesurfer import BBRegister
from nipype.interfaces.ants import Registration


# Settings

In [26]:
curr_dir = os.getcwd()
main_dir = os.pardir
data_dir = os.path.join(main_dir, 'data')


subjects_list = ['sub-10159', 'sub-50004']

FSLCommand.set_default_output_type('NIFTI_GZ')
# location of template file
template = Info.standard_image('MNI152_T1_2mm.nii.gz')

# Load data

In [8]:
from nipype.pipeline.engine import MapNode, Node, Workflow
from nipype.interfaces.io import DataSink, DataGrabber
from nipype.interfaces.utility import IdentityInterface, Function

# Preprocess data

In [23]:
# Create input node. This is where you are specify where you data comes from.
infosource = Node(interface=IdentityInterface(fields=['subject_id']), name='InfoSource')
infosource.iterables = ('subject_id', subjects_list)

datasource = Node(interface=DataGrabber(infields=['subject_id'],
                                       outfields=['anat', 'rest', 'task']), name='datasource')
datasource.inputs.base_directory = main_dir
datasource.inputs.template = '*'
datasource.inputs.sort_filelist = True
datasource.inputs.field_template = dict(rest=os.path.join('%s', 'func', '%s_task_rest_bold.nii.gz'), 
                                        task=os.path.join('%s', 'func', '%s_task_taskswitch_bold.nii.gz'),
                                        anat=os.path.join('%s', 'anat', '%s_T1w.nii.gz'))
datasource.inputs.template_args = dict(rest=[['subject_id', 'subject_id']],
                                       task=[['subject_id', 'subject_id']],
                                       anat=[['subject_id', 'subject_id']])
datasource.inputs.subject_id = subjects_list

In [27]:
# Create data sink. This is where your data is going to be saved
data_sink = Node(DataSink(), name='DataSink')
data_sink.inputs.base_directory = os.path.join(data_dir, 'preprocessing_out')
substitutions = [('_subject_id', ''),
                 ('_warpall0', 'warpall')]

# Node 0: Motion correction
mot_par = Node(MCFLIRT(), name='motion_correction')

# Node:
# Corregister the median to surface
bbreg = Node(BBRegister(), name='bbRegister')
bbreg.inputs.init = 'fsl'
bbreg.inputs.contrast_type = 't2'
bbreg.inputs.out_fsl_file = True
bbreg.inputs.subjects_dir = data_dir


# Node 1: Generate Mean Image
# Generate mean image - only for the EPI image
mean_image = Node(MeanImage(), name='Mean_Image')
    
# Node 2: Registration: T1 - MNI
# parameters from:
# http://miykael.github.io/nipype-beginner-s-guide/normalize.html
antsreg = Node(Registration(), name='antsreg')
antsreg.inputs.args = '--float'
# antsreg.inputs.collapse_output_transforms = True
antsreg.inputs.fixed_image = template
antsreg.inputs.initial_moving_transform_com = True
antsreg.inputs.num_threads = 8
antsreg.inputs.output_warped_image = True
antsreg.inputs.output_inverse_warped_image = True
antsreg.inputs.sigma_units = ['vox']*3
antsreg.inputs.transforms = ['Rigid', 'Affine', 'SyN']
antsreg.inputs.terminal_output = 'file'
antsreg.inputs.winsorize_lower_quantile = 0.005
antsreg.inputs.winsorize_upper_quantile = 0.995
antsreg.inputs.convergence_threshold = [1e-06]
antsreg.inputs.metric = ['MI', 'MI', 'CC']
antsreg.inputs.metric_weight = [1.0]*3
antsreg.inputs.number_of_iterations = [[1000, 500, 250, 100],
                                       [1000, 500, 250, 100],
                                       [100,   70,  50,  20]]
antsreg.inputs.radius_or_number_of_bins = [32,32,4]
antsreg.inputs.sampling_percentage = [0.25, 0.25, 1]
antsreg.inputs.sampling_strategy = ['Regular', 'Regular', 'None']
antsreg.inputs.shrink_factors = [[8,4,2,1]]*3
antsreg.inputs.smoothing_sigmas = [[3, 2, 1, 0]]*3
antsreg.inputs.transform_parameters = [(0.1,), (0.1,), (0.1, 3.0, 0.0)]
antsreg.inputs.use_histogram_matching = True
antsreg.inputs.write_composite_transform = True
antsreg.inputs.save_state = 'savestate.mat'




# Node 3: Rest - MNI

In [30]:
# Specify input and output Node
# ------------------------------
data_sink.inputs.substitutions = substitutions

# Define workflow name and where output will be saved
preproc = Workflow(name='preprocessing')
preproc.base_dir = data_dir

# Define connection between nodes
preproc.connect([ 
                (infosource, datasource, [('subject_id', 'subject_id')] ),
                # get motion parameters
                (datasource, mot_par,     [('rest', 'in_file')] ),
                (mot_par, data_sink,      [('par_file', 'mcflirt.par_file')] ),
                # get mean image (functional data)
                #(mot_par, mean_image,     [('out_file', 'in_file')]),
                #(infosource, bbreg,       [('subject_id', 'subject_id')]),
                #(mean_image, bbreg,       [('out_file', 'source_file')])
                ])

# Run pipeline
preproc.run()

170926-11:21:18,104 workflow INFO:
	 Workflow preprocessing settings: ['check', 'execution', 'logging']


AttributeError: 'DiGraph' object has no attribute 'out_edges_iter'

In [32]:
import networkx
networkx.__version__


'2.0'